In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
scaler = StandardScaler()
import pandas_profiling
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedKFold
from imblearn.metrics import geometric_mean_score as geo
from imblearn.metrics import make_index_balanced_accuracy as iba
from sklearn.metrics import roc_curve, auc
from imblearn.metrics import geometric_mean_score, make_index_balanced_accuracy, classification_report_imbalanced
from pycaret.regression import *
import sqlite3
from sqlite3 import Error
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts

In [2]:
df1=pd.read_csv('train.csv')

In [3]:
df2=pd.read_csv('test.csv')

In [4]:
df=df1.append(df2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [6]:
df.isnull().sum().sort_values(ascending=False).head(40)

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
SalePrice        1459
FireplaceQu      1420
LotFrontage       486
GarageFinish      159
GarageQual        159
GarageCond        159
GarageYrBlt       159
GarageType        157
BsmtExposure       82
BsmtCond           82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrType         24
MasVnrArea         23
MSZoning            4
Functional          2
BsmtHalfBath        2
BsmtFullBath        2
Utilities           2
BsmtFinSF1          1
BsmtFinSF2          1
SaleType            1
Electrical          1
GarageCars          1
TotalBsmtSF         1
GarageArea          1
Exterior2nd         1
Exterior1st         1
KitchenQual         1
BsmtUnfSF           1
Fireplaces          0
TotRmsAbvGrd        0
SaleCondition       0
OpenPorchSF         0
YrSold              0
dtype: int64

In [7]:
df=df.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'],axis=1)

In [8]:
df['age']=2022-df['YearBuilt']
df['sold_age']=df['YrSold']-df['YearBuilt']
df['remodel_age']=2022-df['YearRemodAdd']
df=df.drop(columns=['MoSold'])
df['garage_age']=2022-df['GarageYrBlt']

In [9]:
df=df.drop(['YrSold','YearBuilt','YearRemodAdd','GarageYrBlt'],axis=1)

In [10]:
df['LotFrontage'].fillna(df.groupby('Neighborhood')['LotFrontage'].transform('mean'),inplace=True)
df['garage_age'].fillna(df['garage_age'].mean(),inplace=True)
df['TotalBsmtSF'].fillna(df['TotalBsmtSF'].mean(),inplace=True)

df['GarageCars'].fillna(0,inplace=True)
df['GarageArea'].fillna(0,inplace=True)
df['GarageType'].fillna('None',inplace=True)
df['GarageQual'].fillna('None',inplace=True)
df['GarageCond'].fillna('None',inplace=True)
df['GarageFinish'].fillna('Unf',inplace=True)

df['Exterior1st'].fillna('VinylSd',inplace=True)
df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0],inplace=True)

df['BsmtFullBath'].fillna(0,inplace=True)
df['BsmtHalfBath'].fillna(0,inplace=True)
df['BsmtFinType2'].fillna('None',inplace=True)
df['BsmtFinSF1'].fillna(0,inplace=True)
df['BsmtFinSF2'].fillna(0,inplace=True)
df['BsmtUnfSF'].fillna(0,inplace=True)
df['BsmtExposure'].fillna('No',inplace=True)
df['BsmtCond'].fillna('None',inplace=True)
df['BsmtQual'].fillna('None',inplace=True)
df['BsmtFinType1'].fillna('Unf',inplace=True)

df['Utilities'].fillna('AllPub',inplace=True)
df['Electrical'].fillna('SBrkr',inplace=True)
df['KitchenQual'].fillna('TA',inplace=True)
df['Functional'].fillna('Typ',inplace=True)

df['MSZoning'].fillna('RL',inplace=True)
df['SaleType'].fillna('WD',inplace=True)
df['MasVnrArea'].fillna(0,inplace=True)
df['MasVnrArea'].fillna('None',inplace=True)

In [11]:
df_dummies=pd.get_dummies(df,drop_first=True)

In [12]:
abs(df_dummies.corr()['SalePrice']).sort_values(ascending=False)

SalePrice                1.000000
OverallQual              0.790982
GrLivArea                0.708624
GarageCars               0.640409
GarageArea               0.623431
TotalBsmtSF              0.613581
1stFlrSF                 0.605852
ExterQual_TA             0.589044
FullBath                 0.560664
TotRmsAbvGrd             0.533723
sold_age                 0.523350
age                      0.522897
KitchenQual_TA           0.519298
GarageFinish_Unf         0.513906
remodel_age              0.507101
Foundation_PConc         0.497734
MasVnrArea               0.472614
garage_age               0.471062
Fireplaces               0.466929
ExterQual_Gd             0.452466
BsmtQual_TA              0.452394
BsmtFinType1_GLQ         0.434597
Neighborhood_NridgHt     0.402149
BsmtFinSF1               0.386420
MasVnrType_None          0.374468
SaleType_New             0.357509
GarageType_Detchd        0.354141
SaleCondition_Partial    0.352060
LotFrontage              0.348056
Foundation_CBl

In [13]:
df_train=df_dummies[:1459]
df_test=df_dummies[1459:]

In [15]:
y=df_train['SalePrice']
x=df_train.drop(['SalePrice'],axis=1)

In [16]:
x_train,x_test,y_train,y_test=tts(x,y,test_size=0.2)

In [17]:
GBR=GradientBoostingRegressor()
GBR.fit(x_train,y_train)
y_pred_gbr=GBR.predict(x_test)

In [18]:
exp_name=setup(data=df,target='SalePrice',fold_shuffle=True)

,Description,Value
0,session_id,7151
1,Target,SalePrice
2,Original Data,"(2919, 75)"
3,Missing Values,True
4,Numeric Features,26
5,Categorical Features,48
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1010, 318)"


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [19]:
best_model=compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,15471.6985,795974768.5418,27385.7137,0.8680,0.1241,0.0874,2.2580
gbr,Gradient Boosting Regressor,16804.7035,780544901.2562,27286.1372,0.8624,0.1383,0.0996,0.1380
lightgbm,Light Gradient Boosting Machine,17952.1151,966871671.6993,30529.6155,0.8369,0.1454,0.1030,0.1600
xgboost,Extreme Gradient Boosting,18330.4646,949211187.2000,30334.1355,0.8350,0.1431,0.1027,0.2660
rf,Random Forest Regressor,18179.6749,1020901802.4628,31162.9440,0.8266,0.1509,0.1070,0.3240
ridge,Ridge Regression,19106.8604,1332655507.2000,34227.9385,0.7694,0.1631,0.1127,0.0250
et,Extra Trees Regressor,21643.9525,1416097757.8731,36747.7801,0.7601,0.1703,0.1226,0.3490
ada,AdaBoost Regressor,25747.8764,1551299203.1863,38462.0516,0.7374,0.2058,0.1666,0.0980
lasso,Lasso Regression,19253.1270,1432316345.6000,35518.6145,0.7372,0.2305,0.1136,0.2490
omp,Orthogonal Matching Pursuit,18538.4487,1450926500.3988,35101.4614,0.7319,0.1644,0.1072,0.0210


In [20]:
X=pd.get_dummies(df_test,drop_first=True)

In [21]:
del X ['SalePrice']

In [22]:
predictions=GBR.predict(X)

In [23]:
X.loc[:,'SalePrice']=predictions

In [26]:
output=pd.DataFrame({'Id':df_test.Id,'SalePrice':predictions})

In [27]:
output.to_csv('predicted_house_sale_price.csv')